In [ ]:
Recurrent Neural Networks (RNNs)

#drawbacks
vanishing gradient
exploring gradient 

we use LSTM (long short term memory)
it use gating mechanism input gate , forget gate , output gate 


GRU algorithm ()
update gate 
reset gate
condidate activation (it combine current state with reset state)
final hidden state




In [1]:
import torch
import torch.nn as nn
import torchtext
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import random

# Load and preprocess
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Encode text
def encode(text):
    return vocab(tokenizer(text))

# Define collate function
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(torch.tensor(encode(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels), texts

# Model
class GRUModel(nn.Module):
    def _init_(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0))

# Training
def train_model():
    model = GRUModel(len(vocab), 64, 128, 1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_iter = IMDB(split='train')
    dataloader = DataLoader(list(train_iter), batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        for labels, texts in dataloader:
            preds = model(texts).squeeze()
            loss = criterion(preds, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

train_model()

ModuleNotFoundError: No module named 'torch'

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/215.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/215.1 MB 2.8 MB/s eta 0:01:17
   ---------------------------------------- 1.0/215.1 MB 3.1 MB/s eta 0:01:09
   ---------------------------------------- 1.6/215.1 MB 2.9 MB/s eta 0:01:14
   ---------------------------------------- 2.4/215.1 MB 2.8 MB/s eta 0:01:17
    --------------------------------------- 2.9/215.1 MB 2.8 MB/s eta 0:01:18
    --------------------------------------- 3.4/215.1 MB 2.7 MB/s eta 0:01:18
    --------------------------------------- 3.7/215.1 MB 2.6 MB/s eta 0:01:22
    --------------------------------------- 4.5/215.1 MB 2.6 MB/s eta 0:01:21
    --------------------------------------- 4.5/215.1 MB 2.6 MB/s eta 0:01:21
    --------------------------------------- 5.0/215.1 MB 2.3 MB/s eta 0:01:31
   - ---------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ------------------------------------ --- 195.8/215.1 MB 2.9 MB/s eta 0:00:07
   ------------------------------------ --- 196.9/215.1 MB 2.9 MB/s eta 0:00:07
   ------------------------------------ --- 197.7/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------ --- 198.2/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------ --- 198.7/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------- -- 199.2/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------- -- 199.8/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------- -- 200.3/215.1 MB 2.9 MB/s eta 0:00:06
   ------------------------------------- -- 200.8/215.1 MB 2.9 MB/s eta 0:00:05
   ------------------------------------- -- 201.3/215.1 MB 2.9 MB/s eta 0:00:05
   ------------------------------------- -- 202.1/215.1 MB 2.9 MB/s eta 0:00:05
   ------------------------------------- -- 202.6/215.1 MB 2.9 MB/s eta 0:00:05
   -------------------------------------

In [6]:
!pip install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cpu

import torch
import torch.nn as nn
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import random

# Tokenizer and dataset
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

# Build vocabulary
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Text encoding
def encode(text):
    return vocab(tokenizer(text))

# Collate function for DataLoader
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(torch.tensor(encode(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels), texts

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0))

# Training function
def train_model():
    model = GRUModel(len(vocab), embed_dim=64, hidden_dim=128, output_dim=1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_iter = IMDB(split='train')
    dataloader = DataLoader(list(train_iter), batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        for labels, texts in dataloader:
            preds = model(texts).squeeze()
            loss = criterion(preds, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

# Run training
train_model()


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
     ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
     ---------- ----------------------------- 0.5/1.9 MB 764.3 kB/s eta 0:00:02
     ---------------- ----------------------- 0.8/1.9 MB 799.2 kB/s eta 0:00:02
     ---------------- ----------------------- 0.8/1.9 MB 799.2 kB/s eta 0:00:02
     --------------------- ------------------ 1.0/1.9 MB 839.3 kB/s eta 0:00:02
     -------------------------- ------------- 1.3/1.9 MB 860.9 kB/s eta 0:00:01
     -------------------------------- ------- 1.6/1.9 MB 932.2 kB/s eta 0:00:01
     -------------------------

ModuleNotFoundError: No module named 'torch'

In [8]:
# Install PyTorch and TorchText if not already installed (for Colab or Jupyter)
# !pip install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cpu

import torch
import torch.nn as nn
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import random

# Tokenizer and dataset
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

# Build vocabulary
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Text encoding
def encode(text):
    return vocab(tokenizer(text))

# Collate function for DataLoader
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(torch.tensor(encode(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels), texts

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0))

# Training function
def train_model():
    model = GRUModel(len(vocab), embed_dim=64, hidden_dim=128, output_dim=1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_iter = IMDB(split='train')
    dataloader = DataLoader(list(train_iter), batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        for labels, texts in dataloader:
            preds = model(texts).squeeze()
            loss = criterion(preds, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

# Run training
train_model()


ModuleNotFoundError: No module named 'torch'

In [7]:
import torch
import torch.nn as nn
import torchtext
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import random

# Load and preprocess
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Encode text
def encode(text):
    return vocab(tokenizer(text))

# Define collate function
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(torch.tensor(encode(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels), texts

# Model
class GRUModel(nn.Module):
    def _init_(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self)._init_()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0))

# Training
def train_model():
    model = GRUModel(len(vocab), 64, 128, 1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_iter = IMDB(split='train')
    dataloader = DataLoader(list(train_iter), batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        for labels, texts in dataloader:
            preds = model(texts).squeeze()
            loss = criterion(preds, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

train_model()

OSError: [WinError 127] The specified procedure could not be found

In [4]:
# If running in Colab or Jupyter, uncomment the line below to install dependencies:
!pip install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cpu

import torch
import torch.nn as nn
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import random
import numpy as np

# Set random seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer and dataset
tokenizer = get_tokenizer('basic_english')
train_iter = list(IMDB(split='train'))  # Convert iterator to list for multiple passes

# Build vocabulary
def yield_tokens(data_iter):
    for label, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Text encoding
def encode(text):
    return vocab(tokenizer(text))

# Collate function for DataLoader
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(1 if label == 'pos' else 0)
        texts.append(torch.tensor(encode(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels, dtype=torch.float32), texts

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<unk>"])
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0)).squeeze(1)  # Output shape: (batch,)

# Training function
def train_model():
    model = GRUModel(len(vocab), embed_dim=64, hidden_dim=128, output_dim=1).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Reload train data for DataLoader
    train_data = list(IMDB(split='train'))
    dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for labels, texts in dataloader:
            labels, texts = labels.to(device), texts.to(device)
            preds = model(texts)
            loss = criterion(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Calculate accuracy
            predictions = torch.sigmoid(preds) > 0.5
            correct += (predictions.float() == labels).sum().item()
            total += labels.size(0)
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

# Run training
train_model()


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


OSError: [WinError 127] The specified procedure could not be found

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import csv
import random

# Custom tokenizer and dataset implementation
def basic_english_tokenizer(text):
    return text.lower().replace('.', ' ').replace(',', ' ').split()

class TextDataset(Dataset):
    def __init__(self, file_path):
        self.samples = []
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                # CSV format: label (0/1), text
                self.samples.append((int(row[0]), row[1]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]  # (label, text)

# Build vocabulary from training data
train_data = TextDataset('train.csv')  # Replace with your CSV path

all_tokens = []
for label, text in train_data:
    all_tokens.extend(basic_english_tokenizer(text))

# Create vocabulary with <unk> token
vocab = {'<unk>': 0}
for token in all_tokens:
    if token not in vocab:
        vocab[token] = len(vocab)

def text_to_indices(text):
    return [vocab.get(token, 0) for token in basic_english_tokenizer(text)]

# Collate function with padding
def collate_batch(batch):
    labels, texts = [], []
    for label, text in batch:
        labels.append(label)
        texts.append(torch.tensor(text_to_indices(text), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels, dtype=torch.float32), texts

# GRU Model
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0)).squeeze(1)

# Training setup
def train():
    model = GRUClassifier(len(vocab))
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        correct = 0
        total = 0
        for labels, texts in dataloader:
            outputs = model(texts)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch {epoch+1} | Loss: {total_loss/len(dataloader):.4f} | Acc: {correct/total:.4f}")

# Start training
train()


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [10]:
# Install needed packages
# !pip install torch torchtext datasets --quiet

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Load IMDB dataset from HuggingFace
dataset = load_dataset("imdb")
train_data = dataset["train"]

# Tokenizer and Vocabulary
tokenizer = get_tokenizer("basic_english")

def yield_tokens(data_iter):
    for example in data_iter:
        yield tokenizer(example["text"])

vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Encode text
def encode(text):
    return vocab(tokenizer(text))

# Collate function
def collate_batch(batch):
    labels, texts = [], []
    for example in batch:
        labels.append(1 if example["label"] == 1 else 0)
        texts.append(torch.tensor(encode(example["text"]), dtype=torch.int64))
    texts = nn.utils.rnn.pad_sequence(texts, batch_first=True)
    return torch.tensor(labels), texts

# GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h_n = self.gru(x)
        return self.fc(h_n.squeeze(0))

# Training function
def train_model():
    model = GRUModel(len(vocab), 64, 128, 1)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_batch)

    for epoch in range(3):
        total_loss = 0
        for labels, texts in dataloader:
            preds = model(texts).squeeze()
            loss = criterion(preds, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Start training
train_model()


ModuleNotFoundError: No module named 'datasets'